Givens Rotation Implementation circuit based off of: http://arxiv.org/abs/2004.04174

In [12]:
# Create provider for IonQ system
from azure.quantum.cirq import AzureQuantumService
service = AzureQuantumService(
    resource_id="",
    location="West US",
    default_target="ionq.simulator"
)

In [13]:
# Import Cirq and Numpy
import cirq
import numpy as np

In [15]:
class SqrtISWAP(cirq.Gate):
    def __init__(self):
        super(SqrtISWAP, self)

    def _num_qubits_(self):
        return 2
    
    # Obtained from: https://github.com/quantumlib/Cirq/blob/a22269dfe41b0da78243bbd210a915d26cc7d25f/cirq-core/cirq/ops/swap_gates.py#L166
    def _decompose_(self, qubits):
        q0, q1 = qubits
        yield cirq.CX(q0,q1)
        yield cirq.H(q0)
        yield cirq.CX(q1,q0)
        yield cirq.ZPowGate(exponent=0.25).on(q0)
        yield cirq.CX(q1,q0)
        yield cirq.ZPowGate(exponent=-0.25).on(q0)
        yield cirq.H(q0)
        yield cirq.CX(q0,q1)
        
    def _circuit_diagram_info_(self, args):
        return ["sqrt(iSWAP)"] * self.num_qubits()

In [16]:
# Create the full Givens Rotation circuit

## Arbitrarily chosen Theta, should be made parameterizable for Variational algorithms
theta = np.pi/4

# create two qubit circuit
circuit = cirq.Circuit()
q0, q1 = cirq.LineQubit.range(2)

# Have to EXPLICITLY call decomposition, Azure Quantum doesn't seem to auto-decompose
# to IonQ supported gates (refer to the IonQ API for an explicit set of supported gates)
circuit.append(SqrtISWAP().on(q0,q1)._decompose_())

# -theta in fig 1b. of http://arxiv.org/abs/2004.04174
circuit.append(cirq.Rz(rads=-theta).on(q0))
# theta + pi in fig 1b. of http://arxiv.org/abs/2004.04174
circuit.append(cirq.Rz(rads=-theta + np.pi).on(q1))


circuit.append(SqrtISWAP().on(q0,q1)._decompose_())


# pi in fig 1b. of http://arxiv.org/abs/2004.04174
circuit.append(cirq.Rz(rads=np.pi).on(q1))
circuit.append(cirq.measure(q0, q1, key='b')) # Measure both qubits

In [17]:
# visualize circuit
print(circuit)

0: ───@───H───X───T───X───T^-1───H───@───Rz(-0.25π)───@───H───X───T───X───T^-1───H───@───────────M('b')───
      │       │       │              │                │       │       │              │           │
1: ───X───────@───────@──────────────X───Rz(0.75π)────X───────@───────@──────────────X───Rz(π)───M────────


In [18]:
cost = service.estimate_cost(circuit, repetitions=100, target="ionq.qpu")
print("Estimated cost:")
print(cost.estimated_total)

Estimated cost:
1


In [19]:
# OPTIONAL Givens rotation Custom gate implemntation
# Taking the circuit above, use it as the basis for a custom gate definition that will make things more modular
# Builds on the SqrtISWAP class above

class IonSupportedGivens(cirq.Gate):
    def __init__(self, theta):
        super(IonSupportedGivens, self)
        self.theta = theta

    def _num_qubits_(self):
        return 2

    def _decompose_(self, qubits):
        q0, q1 = qubits
        # Have to EXPLICITLY call decomposition, Azure Quantum doesn't seem to auto-decompose
        # to IonQ supported gates
        yield SqrtISWAP().on(q0,q1)._decompose_()
        
        yield cirq.Rz(rads=-self.theta).on(q0)
        yield cirq.Rz(rads=-self.theta + np.pi).on(q1)

        yield SqrtISWAP().on(q0,q1)._decompose_()
        yield cirq.Rz(rads=np.pi).on(q1)
        
    def _circuit_diagram_info_(self, args):
        return ["IonGivens({:.3f})".format(self.theta)] * self.num_qubits()

In [20]:
# Test Ion Supported Givens

# Create new circuit
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit()

circuit.append(cirq.decompose_once(IonSupportedGivens(theta=np.pi/4).on(q0, q1)))
print(circuit)

# If you try not decomposing at least once IonQ will not accept the circuit (complains it is non-serializable due to
# not being decomposable).
# circuit.append(IonSupportedGivens(theta=np.pi/4).on(q0, q1))
# print(circuit)

# If you just use cirq.decompose() over cirq.decompose_once() the decomposition goes too far and introduces
# CZ gates that the IonQ API doesn't seem to support
# circuit.append(cirq.decompose(IonSupportedGivens(theta=np.pi/4).on(q0, q1)))
# print(circuit)


0: ───@───H───X───T───X───T^-1───H───@───Rz(-0.25π)───@───H───X───T───X───T^-1───H───@───────────
      │       │       │              │                │       │       │              │
1: ───X───────@───────@──────────────X───Rz(0.75π)────X───────@───────@──────────────X───Rz(π)───


In [21]:
cost = service.estimate_cost(circuit, repetitions=100, target="ionq.qpu")
print("Estimated cost:")
print(cost.estimated_total)

Estimated cost:
1
